In [ ]:
import os
import glob
import requests

In [ ]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor ,RandomForestRegressor


# **load datasets**

In [ ]:
df1=pd.read_csv("/content/Train_Batch_1.csv")
df2=pd.read_csv("/content/Train_Batch_2.csv")
df3=pd.read_csv("/content/Train_Batch_3.csv")
test=pd.read_csv("/content/test.csv")

# **rename features**

In [ ]:
new_column_names_df2 = {
    'Person Description': 'Person_Description',
    'Place Code': 'Place_Code',
    'Customer Order': 'Customer_Order',
    'Additional Features in market': 'Additional_Features_in_market',
    'Promotion Name': 'Promotion_Name',
    'Store Kind': 'Store_Kind',
    'Store Sales': 'Store_Sales',
    'Store Cost': 'Store_Cost',
    'Gross Weight':'Gross_Weight',
    'Net Weight':'Net_Weight',
    'Is Recyclable?': 'Is_Recyclable?',
    'Min. Person Yearly Income':'Min_Person_Yearly_Income',
    'Store Area': 'Store_Area',
    'Grocery Area': 'Grocery_Area',
    'Frozen Area': 'Frozen_Area',
    'Meat Area': 'Meat_Area',
}
new_column_names_df1 = {
    'Person Description': 'Person_Description',
    'Place Code': 'Place_Code',
    'Customer Order': 'Customer_Order',
    'Additional Features in market': 'Additional_Features_in_market',
    'Promotion Name': 'Promotion_Name',
    'Store Kind': 'Store_Kind',
    'Store Sales': 'Store_Sales',
    'Store Cost': 'Store_Cost',
    'Product Weights Data in (KG)':'Product_Weights_Data_in_(KG)',
    'Is Recyclable?': 'Is_Recyclable?',
    'Min. Yearly Income': 'Min_Person_Yearly_Income',
    'Store Area': 'Store_Area',
    'Grocery Area': 'Grocery_Area',
    'Frozen Area': 'Frozen_Area',
    'Meat Area': 'Meat_Area',
}
new_column_names_df3 = {
    'Person%20Description': 'Person_Description',
    'Place%20Code': 'Place_Code',
    'Place Code':'Place_Code',
    'Customer%20Order': 'Customer_Order',
    'Additional%20Features%20in%20market': 'Additional_Features_in_market',
    'Promotion%20Name': 'Promotion_Name',
    'Store%20Kind': 'Store_Kind',
    'Store%20Sales': 'Store_Sales',
    'Store%20Cost': 'Store_Cost',
    'Gross%20Weight': 'Gross_Weight',
    'Net%20Weight': 'Net_Weight',
    'Weights%20Data': 'Product_Weights_Data_in_(KG)',
    'Is%20Recyclable?': 'Is_Recyclable?',
    'Yearly Income': 'Min_Person_Yearly_Income',
    'Yearly%20Income': 'Min_Person_Yearly_Income',
    'Store%20Area': 'Store_Area',
    'Grocery%20Area': 'Grocery_Area',
    'Frozen%20Area': 'Frozen_Area',
    'Meat%20Area': 'Meat_Area',
    'Package Weight' : 'Pac_Weight'
}

df3=df3.rename(columns=new_column_names_df3)
df2=df2.rename(columns=new_column_names_df2)
df1=df1.rename(columns=new_column_names_df1)
test=test.rename(columns=new_column_names_df1)
test=test.rename(columns=new_column_names_df2)
test=test.rename(columns=new_column_names_df3)

# **funcation to convert data type from string to float**

In [ ]:
def convert_type(x):
  if x!=x:
    return x
  try:
        return float(x)

  except ValueError:
        return 0


In [ ]:
df1['Grocery_Area']=df1['Grocery_Area'].apply(convert_type)
df1['Store_Area']=df1['Store_Area'].apply(convert_type)
df1['Meat_Area']=df1['Meat_Area'].apply(convert_type)
df2['Frozen_Area']=df2['Frozen_Area'].apply(convert_type)
######################################
df1['Grocery_Area']=df1['Grocery_Area'].replace({0:df1['Grocery_Area'].median()})
df1['Store_Area']=df1['Store_Area'].replace({0:df1['Store_Area'].median()})
df1['Meat_Area']=df1['Meat_Area'].replace({0:df1['Meat_Area'].median()})
df2['Frozen_Area']=df2['Frozen_Area'].replace({0:df2['Frozen_Area'].median()})
########################################


In [ ]:
def net_weights(x):
    res = eval(x.replace("'", "\""))
    return res['Net Weight']
def gross_weights(x):
    res = eval(x.replace("'", "\""))
    return res['Gross Weight']

In [ ]:
df1['Net_Weight'] =df1['Product_Weights_Data_in_(KG)'].apply(net_weights)
df1['Gross_Weight'] =df1['Product_Weights_Data_in_(KG)'].apply(gross_weights)
df1['Pac_Weight'] =df1['Gross_Weight']-df1['Net_Weight']
df1=df1.drop(['Product_Weights_Data_in_(KG)'],axis=1)
#####################################################
df2['Pac_Weight'] =df2['Gross_Weight']-df2['Net_Weight']
####################################################
df3['Net_Weight'] =df3['Product_Weights_Data_in_(KG)'].apply(net_weights)
df3['Gross_Weight'] =df3['Product_Weights_Data_in_(KG)'].apply(gross_weights)
df3['Pac_Weight'] =df3['Gross_Weight']-df3['Net_Weight']
df3=df3.drop('Product_Weights_Data_in_(KG)' ,axis=1)

In [ ]:
df_merged =pd.concat([df1, df2 ,df3])


In [ ]:
df_merged['marital_status'] = df_merged['Person_Description'].str.split(' ').str[0].replace({"Married": 1, "Single" :0})
test['marital_status'] = test['Person_Description'].str.split(' ').str[0].replace({"Married": 1, "Single" :0})
#######################################
df_merged['gender'] = df_merged['Person_Description'].str.split(' ').str[1].replace({"Male": 1, "Female" :0})
test['gender'] = test['Person_Description'].str.split(' ').str[1].replace({"Male": 1, "Female" :0})
########################################
df_merged['num_children'] = df_merged['Person_Description'].str.split(' ').str[3].replace({"two": 2 ,"four":4, "three" :3, "one" : 1, "five":5 , "No":0 })
test['num_children'] = test['Person_Description'].str.split(' ').str[3].replace({"two": 2 ,"four":4, "three" :3, "one" : 1, "five":5 , "No":0 })
########################################
df_merged['education'] = df_merged['Person_Description'].str.split(', education: ').str[1].str.split(' working').str[0]
test['education'] = test['Person_Description'].str.split(', education: ').str[1].str.split(' working').str[0]

#######################################
df_merged['working'] = df_merged['Person_Description'].str.split(' working as ').str[1]
test['working'] = test['Person_Description'].str.split(' working as ').str[1]
###################################
df_merged = pd.get_dummies(
    data=df_merged,
    columns=["education" ,"working"]
)

test = pd.get_dummies(
    data=test,
    columns=["education" ,"working"]
)
#######################################


In [ ]:
df_merged=df_merged.drop(['Person_Description'],axis=1)
test=test.drop(['Person_Description'],axis=1)

In [ ]:
df_merged.head()

,Unnamed: 0,Place_Code,Customer_Order,Additional_Features_in_market,Promotion_Name,Store_Kind,Store_Sales,Store_Cost,Is_Recyclable?,Min_Person_Yearly_Income,...,education_bachelors degree,education_graduate degree,education_high school degree,education_partial college,education_partial high school,working_clerical,working_management,working_manual,working_professional,working_skilled manual
0,mc_ID_0,H11go_ZA,"Cleaning Supplies from Household department, O...","['Video Store', 'Florist', 'Ready Food', 'Coff...",Dimes Off,Deluxe,8.76 Millions,4.2924 Millions,recyclable,10K+,...,1,0,0,0,0,0,0,0,1,0
1,mc_ID_1,S04ne_WA,"Snack Foods from Snack Foods department, Order...",NaN,Budget Bargains,Supermarket,6.36 Millions,1.9716 Millions,non recyclable,50K+,...,1,0,0,0,0,0,1,0,0,0
2,mc_ID_2,L05es_CA,"Magazines from Periodicals department, Ordered...",['Florist'],Shelf Emptiers,Supermarket,10.86 Millions,4.4526 Millions,recyclable,30K+,...,0,0,1,0,0,0,0,0,0,1
3,mc_ID_3,S03le_WA,"Dairy from Dairy department, Ordered Brand : C...",['Coffee Bar'],Savings Galore,NaN,1.98 Millions,0.6732 Millions,recyclable,30K+,...,0,0,0,1,0,0,0,0,1,0
4,mc_ID_4,M10da_YU,"Vegetables from Produce department, Ordered Br...","['Bar For Salad', 'Ready Food', 'Coffee Bar', ...",Sale Winners,Deluxe,11.56 Millions,4.9708 Millions,non recyclable,50K+,...,1,0,0,0,0,0,0,0,1,0


# brand feature

In [ ]:
def brand_ (x, brand):
    if x!=x:
        return x
    return brand[brand['Brand']==x]['index'].iloc[0]
df_merged['Brand'] = df_merged['Customer_Order'].str.split(' : ').str[1]
test['Brand'] = test['Customer_Order'].str.split(' : ').str[1]
brand =df_merged[['Brand', 'Cost']].groupby('Brand').mean ().reset_index().sort_values(by='Cost').reset_index()
df_merged["Brand"]=df_merged["Brand"].apply(brand_ , args=[brand])
test["Brand"]=test["Brand"].apply(brand_ , args=[brand])

# store sales , store cost and sales profit

In [ ]:
df_merged['Store_Sales'] = df_merged['Store_Sales'].str.replace('Millions', '')
df_merged['Store_Cost'] = df_merged['Store_Cost'].str.replace('Millions', '')
df_merged['Store_Sales'] = df_merged['Store_Sales'].str.replace('M', '').astype(float)
df_merged['Store_Cost'] = df_merged['Store_Cost'].str.replace('M', '').astype(float)
############################
test['Store_Sales'] = test['Store_Sales'].str.replace(' Millions', '')
test['Store_Cost'] = test['Store_Cost'].str.replace(' Millions', '')
test['Store_Sales'] = test['Store_Sales'].str.replace('M', '').astype(float)
test['Store_Cost'] = test['Store_Cost'].str.replace('M', '').astype(float)

##############################################
df_merged['Sales_profit'] = df_merged['Store_Cost'] - df_merged['Store_Sales']
test['Sales_profit'] = test['Store_Cost'] - test['Store_Sales']


# **Is_Recyclable feature**

In [ ]:
df_merged['Is_Recyclable?'] = df_merged['Is_Recyclable?'].replace({'recyclable':1 ,'non recyclable':0 , 'yes' :1 ,'no' :0})
test['Is_Recyclable?'] = test['Is_Recyclable?'].replace({'recyclable':1 ,'non recyclable':0 , 'yes' :1 ,'no' :0})

### **Min_Person_Yearly_Income**

In [ ]:
df_merged['Min_Person_Yearly_Income'] = df_merged['Min_Person_Yearly_Income'].str.replace('K+', '000').str.replace('+', '').astype(float)
test['Min_Person_Yearly_Income'] = test['Min_Person_Yearly_Income'].str.replace('K+', '000').str.replace('+', '').astype(float)


<ipython-input-30-5b172c4cd5e1>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_merged['Min_Person_Yearly_Income'] = df_merged['Min_Person_Yearly_Income'].str.replace('K+', '000').str.replace('+', '').astype(float)
<ipython-input-30-5b172c4cd5e1>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_merged['Min_Person_Yearly_Income'] = df_merged['Min_Person_Yearly_Income'].str.replace('K+', '000').str.replace('+', '').astype(float)
<ipython-input-30-5b172c4cd5e1>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  test['Min_Person_Yearly_Income'] = test['Min_Person_Yearly_Income'].str.replace('K+', '000').str.replace('+', '').astype(float)
<ipython-input-30-5b172c4cd5e1>:2: FutureWarning: The default value of regex will change from T

** order type and department features **

In [ ]:
df_merged['order'] = df_merged.Customer_Order.str.split(", ").str[0].str.split(' from ').str[0]
df_merged['dep']=df_merged.Customer_Order.str.split(", ").str[0].str.split(' from ').str[1]
test['order'] = test.Customer_Order.str.split(", ").str[0].str.split(' from ').str[0]
test['dep']=test.Customer_Order.str.split(", ").str[0].str.split(' from ').str[1]

In [ ]:
test.columns

Index(['Unnamed: 0', 'Place_Code', 'Customer_Order',
       'Additional_Features_in_market', 'Promotion_Name', 'Store_Kind',
       'Store_Sales', 'Store_Cost', 'Gross_Weight', 'Net_Weight', 'Pac_Weight',
       'Is_Recyclable?', 'Min_Person_Yearly_Income', 'Store_Area',
       'Grocery_Area', 'Frozen_Area', 'Meat_Area', 'marital_status', 'gender',
       'num_children', 'education_bachelors degree',
       'education_graduate degree', 'education_high school degree',
       'education_partial college', 'education_partial high school',
       'working_clerical', 'working_management', 'working_manual',
       'working_professional', 'working_skilled manual', 'Brand',
       'Sales_profit', 'order', 'dep'],
      dtype='object')

In [ ]:
test = pd.get_dummies(
    data=test,
    columns=["dep" ,"order","Store_Kind"]
)

df_merged = pd.get_dummies(
    data=df_merged,
    columns=["dep" ,"order","Store_Kind"]
)

test.drop(["Customer_Order"] ,axis=1)
df_merged.drop(["Customer_Order"], axis=1)

,Unnamed: 0,Place_Code,Additional_Features_in_market,Promotion_Name,Store_Sales,Store_Cost,Is_Recyclable?,Min_Person_Yearly_Income,Store_Area,Grocery_Area,...,order_Side Dishes,order_Snack Foods,order_Specialty,order_Starchy Foods,order_Vegetables,Store_Kind_Deluxe,Store_Kind_Gourmet,Store_Kind_Mid-Size,Store_Kind_Small Grocery,Store_Kind_Supermarket
0,mc_ID_0,H11go_ZA,"['Video Store', 'Florist', 'Ready Food', 'Coff...",Dimes Off,8.76,4.2924,1.0,10000.0,2842.23,2037.64,...,0,0,0,0,0,1,0,0,0,0
1,mc_ID_1,S04ne_WA,NaN,Budget Bargains,6.36,1.9716,0.0,50000.0,2814.95,2049.72,...,0,1,0,0,0,0,0,0,0,1
2,mc_ID_2,L05es_CA,['Florist'],Shelf Emptiers,10.86,4.4526,1.0,30000.0,2192.32,1322.21,...,0,0,0,0,0,0,0,0,0,1
3,mc_ID_3,S03le_WA,['Coffee Bar'],Savings Galore,1.98,0.6732,1.0,30000.0,1974.73,NaN,...,0,0,0,0,0,0,0,0,0,0
4,mc_ID_4,M10da_YU,"['Bar For Salad', 'Ready Food', 'Coffee Bar', ...",Sale Winners,11.56,4.9708,0.0,50000.0,2862.30,1872.19,...,0,0,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6461,mc_ID_6461,C15ho_ZA,"['Florist', 'Coffee Bar', 'Bar For Salad', 'Re...",Shelf Emptiers,7.41,3.6309,0.0,30000.0,2207.28,1564.86,...,0,1,0,0,0,0,0,0,0,0
6462,mc_ID_6462,T02ma_WA,"['Ready Food', 'Bar For Salad', 'Coffee Bar', ...",Discount Frenzy,4.60,2.0700,0.0,70000.0,3145.51,2055.29,...,0,0,0,0,0,1,0,0,0,0
6463,mc_ID_6463,S03le_WA,['Coffee Bar'],NaN,3.52,1.3024,0.0,90000.0,1974.49,1236.07,...,0,0,1,0,0,0,0,0,0,1
6464,mc_ID_6464,S03le_WA,['Coffee Bar'],Best Price Savers,2.19,0.9417,0.0,50000.0,1973.77,1236.07,...,0,0,0,0,0,0,0,0,0,1


# Additional_Features_in_market

In [ ]:
def coffee (x):
    if x!=x:
        return 0
    if 'Coffee Bar' in x:
        return 1
    return 0
def  florist (x) :
    if x!=x:
        return 0
    if 'Florist' in x:
        return 1
    return 0
def ready_food (x):
    if x!=x:
        return 0
    if 'Ready Food' in x:
        return 1
    return 0

def Bar_Fors_salad (x):
     if x!=x:
        return 0
     if 'Bar For Salad' in x:
        return 1
     return 0
def video_store(x):
     if x!=x:
        return 0
     if 'Video Store' in x:
        return 1
     return 0

In [ ]:

additional =[]
from nltk.tokenize import word_tokenize
df_merged['Additional_Features_in_market']=df_merged['Additional_Features_in_market'].str.replace("[", '').str.replace(']','').str.replace("'" , '').str.split(", ")
test['Additional_Features_in_market']=test['Additional_Features_in_market'].str.replace("[", '').str.replace(']','').str.replace("'" , '').str.split(", ")
for i in df_merged['Additional_Features_in_market'].value_counts().index:
    for x in  i:
        if x not in additional:
            additional.append(x)

if "" in additional:
    additional.remove("")

In [ ]:
#############################
df_merged[additional[0]]=df_merged['Additional_Features_in_market'].apply(coffee)
df_merged[additional[1]]=df_merged['Additional_Features_in_market'].apply(florist)
df_merged[additional[2]]=df_merged['Additional_Features_in_market'].apply(ready_food)
df_merged[additional[3]]=df_merged['Additional_Features_in_market'].apply(Bar_Fors_salad)
df_merged[additional[4]]=df_merged['Additional_Features_in_market'].apply(video_store)
#############################
test[additional[0]]=test['Additional_Features_in_market'].apply(coffee)
test[additional[1]]=test['Additional_Features_in_market'].apply(florist)
test[additional[2]]=test['Additional_Features_in_market'].apply(ready_food)
test[additional[3]]=test['Additional_Features_in_market'].apply(Bar_Fors_salad)
test[additional[4]]=test['Additional_Features_in_market'].apply(video_store)

In [ ]:

def Promotion_Name_ (x, Promotion_Name):
    if x!=x:
        return x
    if len(Promotion_Name[Promotion_Name['Promotion_Name']==x]['index'])==0:
        return None
    return Promotion_Name[Promotion_Name['Promotion_Name']==x]['index'].iloc[0]
Promotion_Name =df_merged[["Promotion_Name" , "Cost"]].groupby(by="Promotion_Name").mean ().reset_index().sort_values(by='Cost').reset_index()
df_merged["Promotion_Name"]=df_merged["Promotion_Name"].apply(Promotion_Name_ , args=[Promotion_Name])
test["Promotion_Name"]=test["Promotion_Name"].apply(Promotion_Name_ , args=[Promotion_Name])

# Store_Kind

In [ ]:
test["Net_Weight"].fillna(test["Net_Weight"].median(), inplace = True)
test["Gross_Weight"].fillna(test["Gross_Weight"].median(), inplace = True)
test["Pac_Weight"].fillna(test["Pac_Weight"].median(), inplace = True)
#################################
test['Grocery_Area'].fillna(df_merged['Grocery_Area'].median())
test['Store_Area'].fillna(df_merged['Store_Area'].median())
test['Meat_Area'].fillna(df_merged['Meat_Area'].median())
test['Frozen_Area'].fillna(df_merged['Frozen_Area'].median())
###############################
df_merged["Net_Weight"].fillna(test["Net_Weight"].median(), inplace = True)
df_merged["Gross_Weight"].fillna(test["Gross_Weight"].median(), inplace = True)
df_merged["Pac_Weight"].fillna(test["Pac_Weight"].median(), inplace = True)



In [ ]:

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
# To calculate mean use imputer class
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

In [ ]:
columns=test.describe().columns.tolist()
filter=[]
for i in test.describe().columns.tolist():
  if i not in  df_merged.describe().columns.tolist():
    columns.remove(i)




In [ ]:
train_x = imputer.fit_transform(df_merged[columns])
train_y=imputer.fit_transform(np.array (df_merged["Cost"]).reshape(-1,1))
test = imputer.fit_transform(test[columns])

# **get best hyperparameters**

In [ ]:
"""
model = GradientBoostingRegressor()

parameters = {'learning_rate': [0.01,0.02,0.03],
                  'subsample'    : [0.9, 0.5, 0.2],
                  'n_estimators' : [100,500,1000],
                  'max_depth'    : [4,6,8]
                 }
grid = GridSearchCV(estimator=model, param_grid = parameters, cv = 2, n_jobs=-1)
grid.fit(train_x, train_y)
"""

In [ ]:
x_train,x_test ,y_train , y_test =train_test_split(train_x ,train_y)

In [ ]:
model = GradientBoostingRegressor(learning_rate=0.02, max_depth=8, n_estimators=1000, subsample=0.9,min_samples_leaf=4,min_samples_split=10 )
model.fit(x_train, y_train)

In [ ]:
from sklearn.metrics import mean_squared_error
y_pred_scaled = model.predict(x_test)
rse_scaled = np.sqrt(mean_squared_error(y_test, y_pred_scaled))
print('Root mean square error', rse_scaled)

In [ ]:
"""
# Results from Grid Search
print("\n========================================================")
print(" Results from Grid Search " )
print("========================================================")
print("\n The best estimator across ALL searched params:\n",grid.best_estimator_)
print("\n The best score across ALL searched params:\n",grid.best_score_)
print("\n The best parameters across ALL searched params:\n",
          grid.best_params_)
print("\n ========================================================")
 """

# **test**

In [ ]:
pred=pd.DataFrame(grid.predict(test))
pred=pred.reset_index()
pred.to_csv('output.csv')